In [1]:
import numpy as np
import pandas as pd

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [4]:
movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
tags.head(1)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994


In [6]:
tags=tags.drop(['timestamp', 'userId'], axis=1)
tags.head(1)

,movieId,tag
0,60756,funny


In [7]:
tags.shape

(3683, 2)

In [8]:
movies.shape

(9742, 3)

In [9]:
links.shape

(9742, 3)

In [10]:
movies = pd.merge(movies,links, on='movieId')
movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0


In [11]:
movies.shape

(9742, 5)

In [12]:
df = pd.merge(movies,tags, on='movieId', how='outer')
df.head()

,movieId,title,genres,imdbId,tmdbId,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,pixar
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,pixar
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,fun
3,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,fantasy
4,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,magic board game


In [13]:
df.shape

(11853, 6)

In [14]:
def concat_str(series):
    return ' '.join(series.dropna());

In [15]:
agg_func={
    # 'movieId':'first',
    'title':'first',
    'genres':'first',
    'imdbId':'first',	
    'tmdbId':'first',
    'tag':concat_str
}

In [16]:
df = df.groupby('movieId').agg(agg_func).reset_index()

In [17]:
df.head()

,movieId,title,genres,imdbId,tmdbId,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,pregnancy remake


In [18]:
df['genres'] = df['genres'].apply(lambda x: x.replace('|', ' '))

In [19]:
df['tags'] = df['tag'] + ' ' + df['genres']
df.head()

,movieId,title,genres,imdbId,tmdbId,tag,tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,114709,862.0,pixar pixar fun,pixar pixar fun Adventure Animation Children C...
1,2,Jumanji (1995),Adventure Children Fantasy,113497,8844.0,fantasy magic board game Robin Williams game,fantasy magic board game Robin Williams game A...
2,3,Grumpier Old Men (1995),Comedy Romance,113228,15602.0,moldy old,moldy old Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance,114885,31357.0,,Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,pregnancy remake,pregnancy remake Comedy


In [20]:
df.tags.unique()

array(['pixar pixar fun Adventure Animation Children Comedy Fantasy',
       'fantasy magic board game Robin Williams game Adventure Children Fantasy',
       'moldy old Comedy Romance', ...,
       ' Action Adventure Comedy Fantasy Sci-Fi',
       'anime comedy gintama remaster Action Animation Comedy Sci-Fi',
       ' Action Animation Comedy Fantasy'], dtype=object)

In [21]:
df['tags'] = df['tags'].apply(lambda x: x.lower())

In [22]:
def remove_duplicate_words(string):
    words = string.split()
    unique = set()
    result = []

    for word in words:
        if word not in unique:
            unique.add(word)
            result.append(word)
    return ' '.join(result)

In [23]:
df['tags'] = df['tags'].apply(lambda x: remove_duplicate_words(x))

In [24]:
df.head(10)

,movieId,title,genres,imdbId,tmdbId,tag,tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,114709,862.0,pixar pixar fun,pixar fun adventure animation children comedy ...
1,2,Jumanji (1995),Adventure Children Fantasy,113497,8844.0,fantasy magic board game Robin Williams game,fantasy magic board game robin williams advent...
2,3,Grumpier Old Men (1995),Comedy Romance,113228,15602.0,moldy old,moldy old comedy romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance,114885,31357.0,,comedy drama romance
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,pregnancy remake,pregnancy remake comedy
5,6,Heat (1995),Action Crime Thriller,113277,949.0,,action crime thriller
6,7,Sabrina (1995),Comedy Romance,114319,11860.0,remake,remake comedy romance
7,8,Tom and Huck (1995),Adventure Children,112302,45325.0,,adventure children
8,9,Sudden Death (1995),Action,114576,9091.0,,action
9,10,GoldenEye (1995),Action Adventure Thriller,113189,710.0,,action adventure thriller


In [25]:
df = df.dropna()

In [26]:
df['tmdbId'] = df['tmdbId'].astype(int)

In [27]:
df.drop(['tag','genres'], axis=1, inplace=True)

In [28]:
df.head(10)

,movieId,title,imdbId,tmdbId,tags
0,1,Toy Story (1995),114709,862,pixar fun adventure animation children comedy ...
1,2,Jumanji (1995),113497,8844,fantasy magic board game robin williams advent...
2,3,Grumpier Old Men (1995),113228,15602,moldy old comedy romance
3,4,Waiting to Exhale (1995),114885,31357,comedy drama romance
4,5,Father of the Bride Part II (1995),113041,11862,pregnancy remake comedy
5,6,Heat (1995),113277,949,action crime thriller
6,7,Sabrina (1995),114319,11860,remake comedy romance
7,8,Tom and Huck (1995),112302,45325,adventure children
8,9,Sudden Death (1995),114576,9091,action
9,10,GoldenEye (1995),113189,710,action adventure thriller


In [29]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [30]:
def stem(txt):
    y=[]
    for i in txt.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [31]:
df['tags']=df['tags'].apply(lambda x: stem(x))
df.head()

,movieId,title,imdbId,tmdbId,tags
0,1,Toy Story (1995),114709,862,pixar fun adventur anim children comedi fantasi
1,2,Jumanji (1995),113497,8844,fantasi magic board game robin william adventu...
2,3,Grumpier Old Men (1995),113228,15602,moldi old comedi romanc
3,4,Waiting to Exhale (1995),114885,31357,comedi drama romanc
4,5,Father of the Bride Part II (1995),113041,11862,pregnanc remak comedi


In [32]:
df.set_index('movieId')

,title,imdbId,tmdbId,tags
movieId,,,,
1,Toy Story (1995),114709,862,pixar fun adventur anim children comedi fantasi
2,Jumanji (1995),113497,8844,fantasi magic board game robin william adventu...
3,Grumpier Old Men (1995),113228,15602,moldi old comedi romanc
4,Waiting to Exhale (1995),114885,31357,comedi drama romanc
5,Father of the Bride Part II (1995),113041,11862,pregnanc remak comedi
...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),5476944,432131,action anim comedi fantasi
193583,No Game No Life: Zero (2017),5914996,445030,anim comedi fantasi
193585,Flint (2017),6397426,479308,drama


In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500)

In [34]:
vectors = cv.fit_transform(df['tags']).toarray()

In [35]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
cv.get_feature_names_out()

array(['06', '1970', '250', 'aardman', 'abus', 'act', 'action', 'actress',
       'adam', 'adolesc', 'adulteri', 'adventur', 'affleck', 'africa',
       'al', 'alcohol', 'alfr', 'alien', 'altern', 'amaz', 'america',
       'american', 'amnesia', 'and', 'andi', 'android', 'anim', 'ann',
       'anti', 'apocalyps', 'apocalypt', 'appeal', 'archaeolog', 'arnold',
       'art', 'arthur', 'artifici', 'artist', 'artsi', 'as', 'ass',
       'assassin', 'astair', 'at', 'atmospher', 'austen', 'australia',
       'awesom', 'babi', 'bad', 'bale', 'base', 'basebal', 'basketbal',
       'beat', 'beauti', 'ben', 'best', 'bibl', 'big', 'biopic', 'bird',
       'bittersweet', 'black', 'bleak', 'blind', 'bloodi', 'book', 'bore',
       'box', 'brad', 'british', 'brother', 'bruce', 'brutal', 'build',
       'burton', 'buscemi', 'busi', 'capot', 'captain', 'carel', 'cast',
       'cera', 'cerebr', 'charact', 'charl', 'charliz', 'child',
       'children', 'chri', 'christian', 'christma', 'christoph',
    

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
similarity = cosine_similarity(vectors)

In [39]:
similarity[0]

array([1.        , 0.42857143, 0.26726124, ..., 0.        , 0.26726124,
       0.37796447])

In [40]:
def recommend(movie):
    idx = df[df['title'] == movie].index[0]
    dist = similarity[idx]
    movie_list = sorted(list(enumerate(dist)), reverse=True, key=lambda x: x[1])[1:6]

    for i in movie_list:
        print(df.iloc[i[0]].title)

In [41]:
recommend('Toy Story (1995)')

Antz (1998)
Bug's Life, A (1998)
Adventures of Rocky and Bullwinkle, The (2000)
Emperor's New Groove, The (2000)
Monsters, Inc. (2001)


In [42]:
import pickle

In [156]:
df.head()

,movieId,title,imdbId,tmdbId,tags
0,1,Toy Story (1995),114709,862,pixar fun adventur anim children comedi fantasi
1,2,Jumanji (1995),113497,8844,fantasi magic board game robin william adventu...
2,3,Grumpier Old Men (1995),113228,15602,moldi old comedi romanc
3,4,Waiting to Exhale (1995),114885,31357,comedi drama romanc
4,5,Father of the Bride Part II (1995),113041,11862,pregnanc remak comedi


In [158]:
movie

NameError: name 'movie' is not defined

In [162]:
movie = pickle.load(open('Movies.pkl', 'rb'))

In [166]:
pd.DataFrame(movie)

,movieId,title,imdbId,tmdbId,tags
0,1,Toy Story (1995),114709,862,pixar fun adventur anim children comedi fantasi
1,2,Jumanji (1995),113497,8844,fantasi magic board game robin william adventu...
2,3,Grumpier Old Men (1995),113228,15602,moldi old comedi romanc
3,4,Waiting to Exhale (1995),114885,31357,comedi drama romanc
4,5,Father of the Bride Part II (1995),113041,11862,pregnanc remak comedi
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),5476944,432131,action anim comedi fantasi
9738,193583,No Game No Life: Zero (2017),5914996,445030,anim comedi fantasi
9739,193585,Flint (2017),6397426,479308,drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),8391976,483455,action anim
